In [31]:
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import opinion_lexicon
from nltk.corpus import sentiwordnet as swn

nltk.download("opinion_lexicon")

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /Users/wrst/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

- **Ejercicio 1:** Se quiere desarrollar un evaluador de sentimientos basado en el uso de un recursos lingüístico externo, en concreto, las listas de palabras de opinión que tiene NLTK.

In [12]:
opinion1 = "Visceral, stunning and relentless film making. Dicaprio's Herculean, almost purely physical performance" \
            "and Hardy's wide eyed intensity coupled with the almost overwhelming beauty of the landscape - those " \
            "trees, the natural light, the sun peeking through the clouds, rendered the proceedings down to savage" \
            "poetry. A hypnotic, beautiful, exhausting film."

opinion2 = "I saw this film on Friday. For the first 40 minutes involving spoken dialogue they need not have " \
            "bothered. For me the dialogue was totally unintelligible with grunting, southern states drawl, " \
            "and coarse accent that made it impossible to understand what they were saying."

opinion3 = "It was a idiotic film that produces a magnificent fascination."

In [27]:
def classify(word_tokenized):
    score = 0
    for s in word_tokenized:
        for w in s:
            if w in opinion_lexicon.positive():
                score += 1
            elif w in opinion_lexicon.negative():
                score -= 1
    if score == 0:
        return "neutro"
    elif score > 0:
        return "pos"
    else:
        return "neg"

In [28]:
# Opinión 1
sent_tokenized = PunktSentenceTokenizer().tokenize(opinion1)
word_tokenized = [word_tokenize(s) for s in sent_tokenized]
print(classify(word_tokenized))

pos


In [29]:
# Opinión 2
sent_tokenized = PunktSentenceTokenizer().tokenize(opinion2)
word_tokenized = [word_tokenize(s) for s in sent_tokenized]
print(classify(word_tokenized))

neg


In [30]:
# Opinión 3
sent_tokenized = PunktSentenceTokenizer().tokenize(opinion3)
word_tokenized = [word_tokenize(s) for s in sent_tokenized]
print(classify(word_tokenized))

pos


---
- **Ejercicio 2:** Se pide lo mismo que en el ejercicio anterior, pero en este caso utilizando SentiWordNet (disponible en NLTK). Esta base de datos proporciona valores positivos y negativos para ciertas palabras en un rango entre -1 y 1. Utilizar como entrada las mismas opiniones del ejercicio anterior. 